In [1]:
# Cell 1 - Imports
import pandas as pd
from supervised_neural_network import train_global_model, evaluate, GlobalRecModel
import torch
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Caskroom/miniconda/base/envs/ml-project/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Caskroom/miniconda/base/envs/ml-project/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/Caskroom/miniconda/base/envs/ml-project/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/Caskroom/miniconda/base/envs/ml-project/lib/pytho

In [2]:
# Cell 2 - Load data
df = pd.read_csv("./data/processed_music_data.csv")
num_cols = df.select_dtypes(include=[np.number]).columns.to_list()
# feature_cols = [col for col in num_cols if col != "liked"]
feature_cols = ['danceability', 'energy', 'valence', 'tempo']
df[feature_cols] = df[feature_cols].fillna(0)

In [3]:
# Cell 3 - Filter active users
user_counts = df[df['liked'] == 1].groupby('user_id').size()
x = 100
users_over_x = user_counts[user_counts > x].index
filtered_df = df[df['user_id'].isin(users_over_x)]

print(f"Filtered dataset: {len(filtered_df)} interactions, {filtered_df['user_id'].nunique()} users")

Filtered dataset: 473206 interactions, 1132 users


In [4]:
# Cell 4 - Train model
model, user2idx, song2idx = train_global_model(filtered_df, feature_cols, epochs=5)

Using device: cpu
Starting training with manual bias–variance decomposition...


Epoch 1/5: 100%|██████████| 925/925 [00:09<00:00, 93.54it/s, loss=0.0134] 


Epoch 1 - Avg Loss: 0.1189


Epoch 2/5: 100%|██████████| 925/925 [00:09<00:00, 96.82it/s, loss=0.0667] 


Epoch 2 - Avg Loss: 0.0613


...

Epoch 3 - Avg Loss: 0.0417


Epoch 4/5: 100%|██████████| 925/925 [00:09<00:00, 95.34it/s, loss=0.0234] 


Epoch 4 - Avg Loss: 0.0307


Epoch 5/5: 100%|██████████| 925/925 [00:09<00:00, 96.51it/s, loss=0.0097] 


Epoch 5 - Avg Loss: 0.0234
Saved loss and bias-variance tradeoff plots to ./models/


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# now evaluate
evaluate(model, filtered_df, feature_cols, user2idx, song2idx, device=device)

Evaluating 100 users...
50/100 users done.
100/100 users done.

--- Recommendation Metrics ---
HitRate@10:   0.2300
Precision@10: 0.0230
Recall@10:    0.0009
F1@10:        0.0017


{'HitRate@k': 0.23,
 'Precision@k': 0.023000000000000007,
 'Recall@k': 0.000914165788753054,
 'F1@k': 0.001728698531368762}